<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('data_exploration.html'); document.getElementById('topnv').selectedIndex=2; document.getElementById('botnav').selectedIndex=2;"> Data Exploration </p>



<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('baseline.html'); document.getElementById('topnv').selectedIndex=3; document.getElementById('botnav').selectedIndex=3;"> Baseline Model </p>


<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('lsa.html'); document.getElementById('topnv').selectedIndex=4; document.getElementById('botnav').selectedIndex=4;"> LSA </p>

<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('meta_features.html'); document.getElementById('topnv').selectedIndex=5; document.getElementById('botnav').selectedIndex=5;"> Meta Features </p>


<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('lasso.html'); document.getElementById('topnv').selectedIndex=6; document.getElementById('botnav').selectedIndex=6;"> Regularization: Lasso </p>


<p style="color:#0000EE; text-decoration:underline" onclick="includeContent('final.html'); document.getElementById('topnv').selectedIndex=7; document.getElementById('botnav').selectedIndex=7;"> Final Model </p>










new5